This notebook consolidates several empirical experiments which demonstrate the impact of removing two underlying assumptions in GSM modeling: no capacity constraints and availablity of extraordinary measures in situations of inventory stockouts at every single link within the network. 

We identified the prevalence of the following effects:
* If even at least one link in the network lacks extraordinary measures to compensate for internal stockouts, then the effective SLA at the customer facing stage will lower then the one guaranteed by idealized GSM solution

* The drop in effective SLA is worse for a larger discrepancy between demand stage inventory deviations and its adjacent suppliers. But the negative trend is not linear with the respect to the ration of inventory deviations, but rather goes as square root (or log) of this quantity.

* For the networks where upstream stages have much larger lead times compared to customer facing stages downstream, placing safety stocks in a more spread manner across the network results in a smaller drop in effective SLA then more sparse and concentrated allocations, which are typical solutions of basic GSM optimisation.

* In large deep serial networks, propagating stockouts from upstream tend to be attenuated by the safety stocks at intermediate stages before it reaches customer facing inventory. Typically the effect of propagating stockouts is determined mostly by the most adjacent (or direct) upstream neighbours. 

* If capacity constraints are present for stages with long replenishment periods, recovery from abnormally large and long coupled stockouts at demand stage is prolonged to a large degree.

Directions for further investigation:
* Cascading stockouts in convergent supply chains where several of the multiple suppliers to the same stage can stockout independently

* Study cascading effects when safety stocks are computed taking account of capacity constraints

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

Go to cascading_stockouts directory since sandbox modules are not part of snc package

In [ ]:
cd ~/code/snc/sandbox/meio/gsm/cascading_stockouts

In [ ]:
from cascading_stockouts_experiments import sparse_vs_spread_safety_stocks
from vis_utils import plot_cascading_effect
from num_sim_utils import get_new_stockout_intervals,collect_stockout_intervals

In [ ]:
from snc.meio.gsm.utils import read_supply_chain_from_txt

In [ ]:
from snc.experiment.numerical_simulator import simulate, compute_base_stocks, compute_safety_stocks, compute_replenishment_times

## Cascading stockouts in serial networks

Let's load a simple two stage network

In [ ]:
stages = read_supply_chain_from_txt("basic_serial_network_config.txt")
{stage.id:stage.lead_time for stage in stages.values()}

And define arbitrary GSM policy

In [ ]:
policy = {"Demand":{"s":0,"si":3},"Dist":{"s":3,"si":0}}

Network configuration and policy will result in the following net replenishment times

In [ ]:
compute_replenishment_times(stages,policy)

Now generate demand history from a sequence of independent Poisson random variables

In [ ]:
n=10000 # length of simulation (number of days)
lam = 10 # demand daily mean

np.random.seed(seed=8675309)
demand_history = np.random.poisson(size=n,lam=lam)

We then specify required service level and compute required base stocks from GSM policy

In [ ]:
sla = 0.95 + 0.006 # slight correction to account for base stock level discrtisation
base_stocks = compute_base_stocks(stages,policy,lam,sla)
base_stocks

And corresponding running average safety stocks

In [ ]:
compute_safety_stocks(stages,policy,lam,sla)

We can see that standard deviation of Dist stage inventory dynamics is three times more then the Demand stage

That is all we need to simulate the network under two scenarios:
* Assuming existence of extraordinary measures which effectively decouples Dist and Demand inventories
* Assuming inventory stockouts are propagated downstream and are only compensated at a later date when new replenishments arrive

In [ ]:
indep_inv_histories = simulate(stages,policy,base_stocks,{},demand_history,stockout_stages=[])
casc_inv_histories = simulate(stages,policy,base_stocks,{},demand_history,stockout_stages=None)

Independent inventories should have their effective SLA at the required level

In [ ]:
np.mean(indep_inv_histories["Demand"] >= 0),np.mean(indep_inv_histories["Dist"] >= 0)

How about coupled inventories?

In [ ]:
np.mean(casc_inv_histories["Demand"] >= 0),np.mean(casc_inv_histories["Dist"] >= 0)

Dist stage service level remains at the same level since its supplier (not modeled) is assumed never to stockout.
But Demand stage stockout frequency increased by 1.2%, due to propagating stockouts of the Dist stage.

In [ ]:
plot_cascading_effect(casc_inv_histories,indep_inv_histories,["Dist","Demand"],time_length=n,remove_transient=True)

From the plots above we can clearly see the difference in the steady state inventory dynamics for coupled and decoupled inventories of two adjacent stages.

Clearly in coupled case, stockouts in Dist stage, which are 3 times larger, drug Demand stage inventory downwards as well far below its normal independent deviations.

We can study the effect of varying the lead time of Dist stage (forcing stage) and get the idea of how strongly the stockouts at the demand stage are affected

In [ ]:
f,ax = plt.subplots(3,1,figsize=(12,18),sharex=False)
sla_drops = sparse_vs_spread_safety_stocks(n_buffers=0,ax=ax,plot=True)
ax[0].set_title("Effect of cascading stockouts in presence of no intermediate buffers")

(the left most datasample in all the plots corresponds to simulation under ideal assumptions, i.e. decoupled inventories)

For a range of reasonable lead times for upstream stage (10-30 days) the onset of cascading stockouts in this simple two stage network causes 
* drop in effective SLA by 2-3% from the ideal 95% level (in this case)
* increase in the mean stockout duration from 1 day to 3-5 days
* increase in the mean daily back orders queue length from 2 to 3-8 items

We can try another setup. Lets assume we can have a certain number of intermediate stages all with lead time equal to 1 day between demand stage and supplier. We then going to enumerate all possible GSM policies for these intermediate buffers and see which safety stock allocations "protect" the demand stage the most from the high fluctuations in the supply stage.

In [ ]:
n_buffers = 4 #choose number of intermediate buffers, don't pick more than 6

In [ ]:
f,ax = plt.subplots(3,1,figsize=(12,18),sharex=False)
sla_drops = sparse_vs_spread_safety_stocks(n_buffers=n_buffers,ax=ax,plot=True)
ax[0].set_title("Effect of cascading stockouts in presence of {} intermediate buffers".format(n_buffers))

Each color above represents alternative safety stock allocation and we can see that there is a spread of 1% in the effective SLA depending on the allocations.

We can check which allocation causes minimum drop in SLA for each forcing lead time at the supply stage.

(tuple below show safety stock levels (in items) starting from supply stage on the left, then intermediate buffers and demand stage at the end right)

(remember that lead times for intermediate buffers and demand stage are fixed at 1 day)

In [ ]:
for supply_lead_time in sorted(sla_drops):
    print(supply_lead_time,max(sla_drops[supply_lead_time],key=lambda x:sla_drops[supply_lead_time][x]))

The trend is evident. As forcing lead time increases, more spread safety stock allocations are preferable. However completely uniform allocation (-,5,5,5,5,5) is not the best one at any point, because it is better to have slightly higher variance at the demand stage than at its adjacent buffer stage, like all (-,-,-,-,5,8) allocations.

Now lets look at the worst allocations in presence of cascading stockouts

In [ ]:
for supply_lead_time in sorted(sla_drops):
    print(supply_lead_time,min(sla_drops[supply_lead_time],key=lambda x:sla_drops[supply_lead_time][x]))

Clearly, more sparse allocations are the worst at higher forcing lead times. But notice that the most extreme one (-,0,0,0,0,12) is not the least robust for higher forcing lead times.

The takeaway of these experiments is to question the validity of ideal GSM safety stock allocation, which tends to be very sparse based on concave cost function. If extraordinary measures are absent it might be better to have safety stocks more evenly distributed to buffer against cascading stockouts more reliably.

One more thing to take into account is that presence of more intermediate buffers smoothes the propagating stockouts better. Below is the demo.

In [ ]:
f,ax = plt.subplots(2,1,figsize=(12,12),sharex=False)
for n_buffers in range(5):
    sla_drops = sparse_vs_spread_safety_stocks(n_buffers=n_buffers,plot=False)
    l_times = sorted(sla_drops)
    min_sla_drops = [0]
    max_sla_drops = [0]
    for supply_lead_time in sorted(sla_drops):
        min_sla_drop = max(sla_drops[supply_lead_time].values())
        min_sla_drops.append(min_sla_drop)
        max_sla_drop = min(sla_drops[supply_lead_time].values())
        max_sla_drops.append(max_sla_drop)
    ax[0].plot([3e-1]+l_times,min_sla_drops,"-.b",alpha=0.2)
    ax[0].plot([3e-1]+l_times,min_sla_drops,"o",label="{}".format(n_buffers))
    ax[1].plot([3e-1]+l_times,max_sla_drops,"-.b",alpha=0.2)
    ax[1].plot([3e-1]+l_times,max_sla_drops,"o",label="{}".format(n_buffers))

for i in range(2):
    ax[i].grid(axis="y")
    ax[i].set_xscale("log")
    ax[i].set_xlabel("Supply lead time (days)")
    ax[i].legend(title="Number of intermediate buffers")
    ax[i].set_ylabel("Drop from ideal SLA")

    
ax[0].set_title("Best safety stock allocation")
ax[1].set_title("Worst safety stock allocation")

We can see that even for worst safety stock allocation, more buffers results in lower effective SLA drop from ideal.

One possible implication of the finding above is that stockouts upstream are not amplified but rather attenuated by the presence of more intermediate stages in the network (assuming they hold safety stocks) before it reaches customer facing stage and affects service level.

If this is generally true than cascading stockouts do not pose much of a problem in deep serial networks, if safety stocks are held in a more dispersed manner.

## The onset of capacity constraints

We are going to run a similar experiment as at the begining of previous section but now we will add a capacity constraint on a daily batch size which can be processed by Dist stage.

We will also increase the daily demand variance, to make extreme periods deviate more

In [ ]:
stages = read_supply_chain_from_txt("basic_serial_network_config.txt")
{stage.id:stage.lead_time for stage in stages.values()}

In [ ]:
n=100000
lam = 100

np.random.seed(seed=8675309)
demand_history = np.random.poisson(size=n,lam=lam)

In [ ]:
sla = 0.95+0.001
base_stocks = compute_base_stocks(stages,policy,lam,sla)

capacity_constraints = {}
indep_inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=[])
casc_inv_histories = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=None)

In [ ]:
#now check the effective sla with independent stockouts
np.mean(indep_inv_histories["Demand"] >= 0),np.mean(indep_inv_histories["Dist"] >= 0)

In [ ]:
#now check the effective sla with coupled stockouts
np.mean(casc_inv_histories["Demand"] >= 0),np.mean(casc_inv_histories["Dist"] >= 0)

In [ ]:
plot_cascading_effect(casc_inv_histories,indep_inv_histories,["Dist","Demand"],time_length=n,remove_transient=True)

Simulate same system but with capacity constraint at Dist stage

In [ ]:
capacity_constraints = {"Dist":102}
indep_inv_histories_cap = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=[])
casc_inv_histories_cap = simulate(stages,policy,base_stocks,capacity_constraints,demand_history,stockout_stages=None)

In [ ]:
#now check the effective sla with coupled stockouts
np.mean(indep_inv_histories_cap["Demand"] >= 0),np.mean(indep_inv_histories_cap["Dist"] >= 0)

In [ ]:
#verify stockout frequency against sla
np.mean(casc_inv_histories_cap["Demand"] >= 0),np.mean(casc_inv_histories_cap["Dist"] >= 0)

In [ ]:
plot_cascading_effect(casc_inv_histories_cap,indep_inv_histories_cap,["Dist","Demand"],time_length=n,remove_transient=True)

In presence of capacity constraints, safety stocks level set by ideal GSM is inadequate and results in a significant SLA drop at demand stage (-4.2%)

Let's find one of the coupled stockouts

In [ ]:
#find one coupled stockout and plot it
loc = np.where(casc_inv_histories_cap["Demand"]<-80)[0][10]
print(loc)

window = 200
s = loc-window
e = s+2*window
plt.figure(figsize=(12,8))
for stage_id in casc_inv_histories:
    plt.plot(casc_inv_histories_cap[stage_id][s:e],label="{} stage inventory position".format(stage_id))

plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

window = 100
s = loc-window
e = s+2*window
stage_id = "Demand"
plt.figure(figsize=(12,8))
plt.plot(indep_inv_histories[stage_id][s:e],label="{}: no capacity constraint".format(stage_id))
plt.plot(casc_inv_histories[stage_id][s:e],label="{}: no capacity constraint, cascade".format(stage_id))
plt.plot(casc_inv_histories_cap[stage_id][s:e],label="{}: capacity constraint, cascade".format(stage_id))

plt.ylabel("Inventory position")
plt.xlabel("Day")
plt.grid(axis="y")
plt.legend()

From the second plot above we can see that capacity constrained systems takes longer to recover from the stockout (green vs orange) which corresponds to inventory position rising above zero. As a result total stockout time incereases substantially

In [ ]:
casc_stockouts = collect_stockout_intervals(casc_inv_histories["Demand"])
casc_stockouts_cap = collect_stockout_intervals(casc_inv_histories_cap["Demand"])

indep_stockouts = collect_stockout_intervals(indep_inv_histories["Demand"])
indep_stockouts_cap = collect_stockout_intervals(indep_inv_histories_cap["Demand"])

Overall, without capacity constraints the total percentage increase in stockout time is:

In [ ]:
100*(sum([len(inter) for inter in casc_stockouts.values()])/sum([len(inter) for inter in indep_stockouts.values()]) - 1)

With capacity constraints the total percentage increase in stockout time is:

In [ ]:
100*(sum([len(inter) for inter in casc_stockouts_cap.values()])/sum([len(inter) for inter in indep_stockouts_cap.values()])-1)

We need to implement extended GSM with capacity constraints and see if the above undesirably long stockouts disappear